# Preprocessing with some EDA

In [12]:
# Package Load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Preprocessing
불필요한 열 제거, 특정 ID 변수를 categorical type으로 변경

In [13]:
# 불필요한 열
col_notuse = ['ADID', '노출 ID', 'SSP 입찰ID', 'DSP 입찰ID', 'AX 낙찰ID', 'WUID (웹 유저 ID)', '광고 응답 광고주 도메인','OS 버전 ID']
col_use = ['시각', 'ADID 타입', 'DSP ID', '매체 ID', '애드유닛 ID', '플랫폼', 'OS 종류', '사이즈 ID',
       '환율', '광고 응답 소재 카테고리', '국가코드 ID', 'P1', 'P2', 'P3', 'P4', 'P5',
       'winning']

In [14]:
# # chunk to pickle(train dataset)
# tp = pd.read_csv('dataset_round1/round1_train.csv', iterator=True, chunksize=1000000)
# df = pd.concat(tp, ignore_index=True)
# # df.to_pickle('train_full.pkl')

In [15]:
# # Data Load/save는 pickle format 이용함
# df = pd.read_pickle('train_full.pkl')

전체 데이터 중
* 낙찰 O : 약 200만개
* 낙찰 X : 약 530만개

> binary classification

### 불필요한 열 제거

In [16]:
# df.columns

In [17]:
# for col in df.columns[:-6]:
#     print(f'Unique values of column {col} : {len(df[col].unique())}')

불필요한 열 선정 결과

> Drop cols : ADID, 노출 ID, SSP 입찰ID, DSP 입찰ID, AX 낙찰ID, WUID (웹 유저 ID), 도메인, OS 버전 ID

In [18]:
# df_cols = df.columns.drop(col_notuse)

In [19]:
# df = df[df_cols]

In [20]:
# df.to_pickle('train.pkl') # 4.3s(compressed)

### ID cols to categorical coding

In [21]:
df = pd.read_csv('dataset_round1/round1_train.csv', usecols=col_use) # 34.3s

In [22]:
df_test = pd.read_csv('dataset_round1/round1_test.csv', usecols=df.columns.drop(['P5','winning']))

In [23]:
# DSP ID, 매체 ID, 애드유닛 ID to categorical(from ID to integer)
col_cat = ['DSP ID', '매체 ID', '애드유닛 ID']

In [24]:
from pandas.api.types import CategoricalDtype
from copy import deepcopy

for col in col_cat:
    series_whole = pd.concat([df[col],df_test[col]]).astype('category').cat.codes + 1
    df[col] = series_whole[:len(df)]
    df_test[col] = series_whole[len(df):]

In [25]:
df = df.sort_values(by=['시각','winning'])
df_test = df_test.sort_values(by=['시각'])

In [26]:
df = df.reset_index(drop=True)

In [27]:
# df.to_pickle('train_1.pkl')
# df_test.to_pickle('test_1.pkl')

In [ ]:
# df.to_pickle('train.pkl') 
# df_test.to_pickle('test.pkl') # 1.6s

## EDA

In [ ]:
# Data Load
df_train = pd.read_pickle('train.pkl')
df_train = df_train[df_train.columns.drop(['P5'])] # P5는 사용불가능한 변수이므로 제거

df_test = pd.read_pickle('test.pkl')

In [ ]:
col_price = ['P1','P2','P3','P4']

In [ ]:
df_train.loc[df_train.winning==1.0, col_price].corr() # 가격변수간 상관계수 탐색(낙찰)

In [ ]:
df_train.loc[df_train.winning==0.0, col_price].corr() # 가격변수간 상관계수 탐색(유찰)

In [ ]:
df_train.to_pickle('train.pkl')
df_test.to_pickle('test.pkl')

### To Datetime
일시 및 초단위 시각을 직접 사용하는 것은 현재 Classification 문제에서 벗어난 Time-series analysis의 관점이므로 일부 변수로만 추출해서 진행

In [ ]:
df_train = pd.read_pickle('train.pkl')
df_test = pd.read_pickle('test.pkl')

In [ ]:
df_train.시각 = pd.to_datetime(df_train.시각, format='%Y%m%d%H%M%S')
df_test.시각 = pd.to_datetime(df_test.시각, format='%Y%m%d%H%M%S')

In [ ]:
df_train.to_pickle('train.pkl') 
df_test.to_pickle('test.pkl') # 1.6s

In [ ]:
# Variable Extraction
## 주말여부
df_train['weekend'] = df_train.시각.dt.dayofweek > 4
df_train['weekend'] = df_train['weekend'].astype('int')
df_test['weekend'] = df_test.시각.dt.dayofweek > 4
df_test['weekend'] = df_test['weekend'].astype('int')

In [ ]:
## 시간대
df_train['hour'] = df_train.시각.dt.hour
df_test['hour'] = df_test.시각.dt.hour

In [ ]:
## 요일
df_train['dayofweek'] = df_train.시각.dt.dayofweek
df_test['dayofweek'] = df_test.시각.dt.dayofweek

In [ ]:
df_train.to_pickle('train.pkl')
df_test.to_pickle('test.pkl')

### Visualization
- Kernel Density Estimation plot으로 가격변수의 분포 확인

In [ ]:
df_win = df_train[df_train.winning==1.0]
df_lose = df_train[df_train.winning==0.0]

In [ ]:
# If normalized?
df_norm = deepcopy(df_train)
df_minmax = deepcopy(df_train)

for col in col_price:
    df_norm[col] = (df_norm[col]-df_norm[col].mean())/df_norm[col].std()
    df_minmax[col] = (df_minmax[col]-df_minmax[col].min())/(df_minmax[col].max()-df_minmax[col].min())

In [ ]:
df_norm_win = df_norm[df_train.winning==1.0]
df_norm_lose = df_norm[df_train.winning==0.0]

df_minmax_win = df_minmax[df_train.winning==1.0]
df_minmax_lose = df_minmax[df_train.winning==0.0]

In [ ]:
# Mean-Normalized
fig, axes = plt.subplots(2,2,figsize=(12,12))
for idx, col in enumerate(col_price):
    sns.kdeplot(df_norm_win[col], label='Win', ax=axes[idx//2, idx%2])
    sns.kdeplot(df_norm_lose[col], label='Lose', ax=axes[idx//2, idx%2])
    axes[idx//2,idx%2].legend()
    axes[idx//2,idx%2].set_title(f'KDE Plot for {col}')
plt.suptitle('Mean-Normalized Price Variables', fontsize=15)
plt.show() # 32.3s

In [ ]:
# Minimax-Normalized
fig, axes = plt.subplots(2,2,figsize=(12,12))
for idx, col in enumerate(col_price):
    sns.kdeplot(df_minmax_win[col], label='Win', ax=axes[idx//2, idx%2])
    sns.kdeplot(df_minmax_lose[col], label='Lose', ax=axes[idx//2, idx%2])
    axes[idx//2,idx%2].legend()
    axes[idx//2,idx%2].set_title(f'KDE Plot for {col}')
plt.suptitle('Minimax-Normalized Price Variables', fontsize=15)
plt.show() # 34.8s

#### 기존 분포에서는 낙찰/유찰 간 가격변수의 분포가 잘 구분되지 않음 > 로그변환 이용

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(12,12))
for idx, col in enumerate(col_price):
    sns.kdeplot(np.log1p(df_win[col]), label='Win', ax=axes[idx//2, idx%2])
    sns.kdeplot(np.log1p(df_lose[col]), label='Lose', ax=axes[idx//2, idx%2])
    axes[idx//2,idx%2].legend()
    axes[idx//2,idx%2].set_title(f'KDE Plot for {col}')
plt.suptitle('Log-Transformed Price Variables', fontsize=15)
plt.show()

#### 최종 : P1-P4 데이터에 대해 log1p 변환 실행

In [ ]:
for col in col_price:
    df_train[col] = np.log1p(df_train[col])
    df_test[col] = np.log1p(df_test[col])

In [ ]:
df_train.to_pickle('train.pkl') 
df_test.to_pickle('test.pkl')

## Final Data type / Variable Name setting

In [ ]:
df_train = pd.read_pickle('train.pkl')
df_test = pd.read_pickle('test.pkl')

In [ ]:
colnames = ['Datetime', 'ADID_type', 'DSP_ID', 'Media_ID', 'Adunit_ID', 'Platform', 'OS_type', 'Size_ID',
       'Ex_Rate', 'Category', 'Country_ID', 'P1', 'P2', 'P3', 'P4', 'weekend',
       'hour', 'dayofweek']

In [ ]:
df_train.columns = colnames + ['Class']
df_test.columns = colnames

In [ ]:
col_int = ['ADID_type', 'DSP_ID', 'Media_ID', 'Adunit_ID', 'Platform', 'OS_type', 'Size_ID',
           'Country_ID', 'weekend', 'hour', 'dayofweek']

In [ ]:
# Missing value at Country_ID > treat with Mode
df_train['Country_ID'] = df_train.Country_ID.fillna(df_train.Country_ID.mode()[0])
df_test['Country_ID'] = df_test.Country_ID.fillna(df_test.Country_ID.mode()[0])

In [ ]:
# Set integer columns
df_train[col_int] = df_train[col_int].astype('int64')
df_test[col_int] = df_test[col_int].astype('int64')

In [ ]:
df_train.to_pickle('train.pkl') 
df_test.to_pickle('test.pkl')

### Category Treatment
- 모든 서브카테고리까지 포함하기에는 너무 많은 variable 생성됨</br>
- 특정 Main Category(0~26, 0:NA)에 속하는 여부만 파악

In [ ]:
df_train = pd.read_pickle('train.pkl')
df_test = pd.read_pickle('test.pkl')

In [ ]:
# Category fillna with str NA
df_train.Category = df_train.Category.fillna('NA')
df_test.Category = df_test.Category.fillna('NA')

In [ ]:
# Category treatment
from tqdm import tqdm
import math

cat_train = []
cat_test = []

for i in tqdm(range(len(df_train))):
    cat_train.append(df_train.Category[i].split('%2C'))

print(len(cat_train))

for i in tqdm(range(len(df_test))):
    cat_test.append(df_test.Category[i].split('%2C'))

print(len(cat_test))

In [ ]:
st = 'IAB91-38'
int(st[3:st.find('-')])

In [ ]:
st.find('-')

In [ ]:
cat_train_treated = [] # for train
for ls in tqdm(cat_train):
    ls_treated = []
    for item in ls:
        if item == 'NA':
            ls_treated.append(0)
        else:
            if item.find('-') == -1:
                ls_treated.append(int(item[3:]))
            else:
                ls_treated.append(int(item[3:item.find('-')]))
    
    cat_train_treated.append(list(set(ls_treated))) # duplicate remove

print(len(cat_train_treated))

In [ ]:
cat_test_treated = [] # for test
for ls in tqdm(cat_test):
    ls_treated = []
    for item in ls:
        if item == 'NA':
            ls_treated.append(0)
        else:
            if item.find('-') == -1:
                ls_treated.append(int(item[3:]))
            else:
                ls_treated.append(int(item[3:item.find('-')]))
    
    cat_test_treated.append(list(set(ls_treated))) # duplicate remove

print(len(cat_test_treated))

In [ ]:
# To dummy Variable (as IAB_n)
for i in range(27):
    colname_i = 'IAB_' + str(i)
    df_train[colname_i] = 0
    df_test[colname_i] = 0

In [ ]:
# For Train data
for idx, ls in tqdm(enumerate(cat_train_treated)):
    for item in ls:
        colname_i = 'IAB_' + str(item)
        df_train.iloc[idx, df_train.columns.get_loc(colname_i)] = 1

In [ ]:
# For Test data
for idx, ls in tqdm(enumerate(cat_test_treated)):
    for item in ls:
        colname_i = 'IAB_' + str(item)
        df_test.iloc[idx, df_test.columns.get_loc(colname_i)] = 1

In [ ]:
df_train = df_train[df_train.columns.drop(['Category'])]
df_test = df_test[df_test.columns.drop(['Category'])]

In [ ]:
df_train = df_train[df_train.columns.drop(['Class']).tolist()+['Class']] # Class 맨뒤로 보내기

In [ ]:
df_train.to_pickle('train.pkl') 
df_test.to_pickle('test.pkl')